In [ ]:
def espresso_deltadelta(dabest_object, ES, volume_unit = 'nanoliter'):
    
    ## generate the column label to access the data in count or volume dabest_object.data objects
    if ES == 'Count':
        label = 'Total\nFeed Count\nPer Fly'
        
    elif ES == 'Volume':
        if volume_unit == "microliter":
            label = 'Total\nFeed Volume\nPer Fly (µl)'
            
        if volume_unit == "nanoliter":    
            label = 'Total\nFeed Volume\nPer Fly (nl)'
            
        if volume_unit == "centiliter":
            label = 'Total\nFeed Volume\nPer Fly (cl)'
            
        if volume_unit == "milliliter":    
            label = 'Total\nFeed Volume\nPer Fly (ml)'
            
        if volume_unit == "picoliter":
            label = 'Total\nFeed Volume\nPer Fly (pl)'
            
    ## calculate delta metrics using raw data SD and count first
    delta_df = {'id':[],'sd':[], 'N':[]}
    for i in dabest_object.mean_diff.idx:
        # fetch the N and SD of the
        ctrl_raw_N = dabest_object.data[dabest_object.data['plot_groups_with_contrast'] == i[0]][label].count()
        exp_raw_N = dabest_object.data[dabest_object.data['plot_groups_with_contrast'] == i[1]][label].count()
        
        ctrl_raw_SD = dabest_object.data[dabest_object.data['plot_groups_with_contrast'] == i[0]][label].std()
        exp_raw_SD = dabest_object.data[dabest_object.data['plot_groups_with_contrast'] == i[1]][label].std()
        
        # https://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_confidence_intervals/bs704_confidence_intervals5.html
        SD_pooled_delta = np.sqrt(((ctrl_raw_N-1) * (ctrl_raw_SD**2) + (exp_raw_N-1)*(exp_raw_SD**2)) / float((ctrl_raw_N+exp_raw_N-2)))

        delta_df['sd'].append(SD_pooled_delta)
        delta_df['id'].append(i[0] + ' minus ' + i[1])
        delta_df['N'].append(ctrl_raw_N + exp_raw_N)

    ## now calculate the delta-delta metrics
    delta_delta_df = {'id':[],'sd_pooled':[], 'delta1_count':[], 'delta2_count':[], 'ES':[]}
    for i in range(len(delta_df['id'])-1):

        # get the counts and stds for the first and second deltas
        first_delta_count = delta_df['N'][i]
        first_delta_sd = delta_df['sd'][i]

        second_delta_count = delta_df['N'][i+1]
        second_delta_sd = delta_df['sd'][i+1]

        # calculate pooled std between a control and test group
        # https://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_confidence_intervals/bs704_confidence_intervals5.html
        SD_pooled_delta_delta = np.sqrt(((first_delta_count-1) * (first_delta_sd**2) + (second_delta_count-1)*(second_delta_sd**2)) / float((first_delta_count+second_delta_count-2)))

        # store the pooled std and total N along with the names of the control and test
        delta_delta_df['id'].append(delta_df['id'][i+1] + '\n' + 'minus' + '\n' + delta_df['id'][i])
        delta_delta_df['sd_pooled'].append(SD_pooled_delta_delta)
        delta_delta_df['delta1_count'].append(first_delta_count)
        delta_delta_df['delta2_count'].append(second_delta_count)

        # get the delta effect sizes from the dabest object
        ES = dabest_object.mean_diff.statistical_tests['difference'][i+1] - dabest_object.mean_diff.statistical_tests['difference'][i]
        delta_delta_df['ES'].append(ES)

        i = i+1
    
    ## calculate CI bars for delta-delta ES
    delta_delta_df['MOE'] = []
    for i in range(len(delta_delta_df['ES'])):
        delta_delta_df['MOE'].append((1.96 * delta_delta_df['sd_pooled'][i]) * np.sqrt((1/delta_delta_df['delta1_count'][i]) + (1/delta_delta_df['delta2_count'][i])))
    
    ## plot the delta-delta ES
    from matplotlib import pyplot as plt
    import seaborn as sns
    
    # adjust the figure size
    fig = plt.figure(figsize=(10,6))
    ax1 = fig.add_subplot(111)

    for i in range(len(delta_delta_df['ES'])):
        ES = delta_delta_df['ES'][i]
        MOE = delta_delta_df['MOE'][i]
        ID = delta_delta_df['id'][i]
        
        # set the dot and 95CI line color and size (e.g. k is for black)
        ax1.plot(ID, ES, 'ko', markersize = 12)
        ax1.plot([ID, ID], [ES-MOE, ES+MOE], 'k-', linewidth = 3)

    # change the x and y labels
    plt.ylabel('delta-delta ' + label)
    plt.xlabel(' ')
    plt.title(' ')
    plt.axhline(y=0, color = 'k', linewidth = 1)
    
    ## set the y-axis limit here: turn off the autoscale line, and set plt.ylim
    ax1.autoscale(enable=True, axis='y', tight=None)
#     plt.ylim(-5, 10)
    sns.set_style("white")
    sns.despine()
    
    return(fig, pd.DataFrame.from_dict(delta_delta_df))

In [ ]:
## To plot Count delta-deltas, pass the count object: feed_count_dabest
figure, deltadelta_results = espresso_deltadelta(feed_count_dabest, ES = 'Count')
deltadelta_results
# to save the figure as .pdf
# figure.savefig("/Users/Tayfun/Downloads/test.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
## To plot Volume delta-deltas, pass the volume object: feed_vol_dabest
## The volume_unit variable has to be THE SAME AS what you used when calculating the feed_vol_dabest object

figure, deltadelta_results = espresso_deltadelta(feed_vol_dabest, ES = 'Volume', volume_unit = 'nanoliter')
deltadelta_results
# to save the figure as .pdf
# figure.savefig("/Users/Tayfun/Downloads/test.pdf", dpi=1000, bbox_inches='tight')